<a href="https://colab.research.google.com/github/tasinfrancesco/Practical_ML_PSL/blob/main/IASO_BigData2025_intro_to_SparkRDD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# =========================
# Colab: PySpark classic (RDD-friendly) + remove Spark Connect conflict
# =========================

# 0) Remove the package that forces pyspark[connect] ~= 4.0.0
!pip -q uninstall -y dataproc-spark-connect google-cloud-dataproc-spark-connect || true  # package name can vary

# 1) Install Java + pinned PySpark (classic)
!apt-get update -qq
!apt-get install -y openjdk-17-jdk-headless -qq
!pip -q install pyspark==3.5.3

# 2) Configure env + start Spark
import os, subprocess
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-17-openjdk-amd64"
os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]

from pyspark.sql import SparkSession
spark = (SparkSession.builder
         .master("local[*]")
         .appName("colab-rdd")
         .config("spark.ui.showConsoleProgress", "false")
         .getOrCreate())

sc = spark.sparkContext
print("Spark version:", spark.version)
print(subprocess.check_output(["java","-version"], stderr=subprocess.STDOUT).decode().splitlines()[0])

# 3) Quick RDD check
rdd = sc.parallelize([1,2,3,4,5], 2).map(lambda x: (x, x*x))
print(rdd.collect())

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 4.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.5/200.5 kB 13.8 MB/s eta 0:00:00
Spark version: 3.5.3
openjdk version "17.0.17" 2025-10-21
[(1, 1), (2, 4), (3, 9), (4, 16), (5, 25)]
